# 🌾 Grain Segmentation & YOLO Labeling Pipeline

This notebook implements an interactive workflow for grain (or similar object) segmentation
and YOLO label generation.

It is organized into four parts:

1. **Manual single-threshold segmentation** – simple playground.
2. **Manual multi-threshold stability map** – interactive multi-th segmentation (no optimisation).
3. **Stability map + local optimisation** – same as 2, with a button to search good `(t_min, t_max)`.
4. **Full automatic optimisation & labeling** – batch mode, no UI.


In [1]:
# 0. Imports, configuration & shared helpers

import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import (
    IntSlider, Checkbox, Button, HBox, VBox, Dropdown, Output, HTML
)
from IPython.display import display

# ------------------ CONFIG ------------------
DATASET_DIR = "test"   # <- change this if needed
IMAGE_DIR = os.path.join(DATASET_DIR, "images")
LABEL_DIR = os.path.join(DATASET_DIR, "labels")
os.makedirs(LABEL_DIR, exist_ok=True)

# Accepted image extensions
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
# -------------------------------------------

# List all images in dataset
image_paths = [
    p for p in glob.glob(os.path.join(IMAGE_DIR, "*.*"))
    if os.path.splitext(p)[1].lower() in IMG_EXTS
]
if not image_paths:
    raise RuntimeError(f"No images found in {IMAGE_DIR}")

image_names = [os.path.basename(p) for p in image_paths]

# Globals used to store the *last* segmentation result (for YOLO save)
current_stats = None
current_params = None
current_image_shape = None  # (H, W)
current_image_name = None

# Shared text log
log_out = Output()


def _sanitize_params(bg_ksize: int, blur_k: int,
                     t_min: int = None, t_max: int = None,
                     n_steps: int = None, min_hits: int = None):
    """Utility to constrain kernel sizes and threshold parameters."""
    # odd kernel sizes, and >= minimal values
    if bg_ksize % 2 == 0:
        bg_ksize += 1
    if blur_k % 2 == 0:
        blur_k += 1
    if bg_ksize < 3:
        bg_ksize = 3
    if blur_k < 1:
        blur_k = 1

    if t_min is not None and t_max is not None:
        if t_min < 0:
            t_min = 0
        if t_max > 255:
            t_max = 255
        if t_min >= t_max:
            t_max = min(255, t_min + 1)

    if n_steps is not None:
        if n_steps < 1:
            n_steps = 1

    if min_hits is not None and n_steps is not None:
        if min_hits < 1:
            min_hits = 1
        if min_hits > n_steps:
            min_hits = n_steps

    return bg_ksize, blur_k, t_min, t_max, n_steps, min_hits


def _preprocess_gray(gray: np.ndarray, bg_ksize: int, blur_k: int) -> np.ndarray:
    """Background correction + blur on a grayscale image."""
    bg = cv2.medianBlur(gray, bg_ksize)
    gray_norm = cv2.subtract(gray, bg)
    gray_blur = cv2.GaussianBlur(gray_norm, (blur_k, blur_k), 0)
    return gray_blur


def save_bboxes_callback(_):
    """Save YOLO labels for the last segmented image (uses current_* globals)."""
    global current_stats, current_params, current_image_shape, current_image_name

    if current_stats is None or current_image_shape is None or current_image_name is None:
        with log_out:
            print("⚠️ Run a segmentation at least once before saving.")
        return

    H, W = current_image_shape
    stats = current_stats
    params = current_params or {}

    min_area = params.get("min_area", 0)
    max_area = params.get("max_area", float("inf"))

    stem, _ = os.path.splitext(current_image_name)
    label_path = os.path.join(LABEL_DIR, stem + ".txt")

    kept = 0
    with open(label_path, "w") as f:
        for i in range(1, stats.shape[0]):  # skip background
            x, y, w, h, area = stats[i]
            if area < min_area or area > max_area:
                continue
            kept += 1

            cx = x + w / 2.0
            cy = y + h / 2.0

            x_norm = cx / W
            y_norm = cy / H
            w_norm = w / W
            h_norm = h / H

            # Single-class (0) YOLO annotation
            f.write(f"0 {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    with log_out:
        print(f"✅ Saved {kept} boxes to {label_path}")


display(HTML("<h2>🌾 Grain Segmentation & YOLO Labeling Pipeline</h2>"))
display(log_out)


HTML(value='<h2>🌾 Grain Segmentation & YOLO Labeling Pipeline</h2>')

Output()

In [2]:
# 1️⃣ Manual single-threshold segmentation (simple playground)

# ---------- Core single-threshold segmentation ----------

def segment_single_threshold(gray,
                             thresh: int,
                             bg_ksize: int,
                             blur_k: int,
                             invert: bool,
                             min_area: int,
                             max_area: int):
    """Simple single-threshold segmentation pipeline.
    Returns: mask_clean, stats, centroids, kept_count
    """
    H, W = gray.shape[:2]

    bg_ksize, blur_k, _, _, _, _ = _sanitize_params(bg_ksize, blur_k)
    gray_blur = _preprocess_gray(gray, bg_ksize, blur_k)

    # Manage inversion
    val = 255 - gray_blur if invert else gray_blur

    # Simple binary threshold
    _, mask = cv2.threshold(val, int(thresh), 255, cv2.THRESH_BINARY)

    # Morphology to clean mask
    kernel = np.ones((3, 3), np.uint8)
    mask_clean = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel, iterations=1)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        mask_clean, connectivity=8
    )

    kept_count = 0
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area or area > max_area:
            continue
        kept_count += 1

    return mask_clean, stats, centroids, kept_count


# ---------- UI (Part 1) ----------

out_part1 = Output()

# Controls / image selection
image_dropdown_1 = Dropdown(options=image_names, description="Image")

# A. Preprocessing
bg_ksize_1 = IntSlider(value=31, min=3, max=101, step=2, description="BG ksize")
blur_k_1   = IntSlider(value=3,  min=1, max=31,  step=2, description="Blur k")
invert_1   = Checkbox(value=False, description="Invert")

# B. Thresholding
thresh_1   = IntSlider(value=128, min=0, max=255, step=1, description="thresh")

# C. Object filtering
min_area_1 = IntSlider(value=50,   min=1,  max=5000,  step=10, description="Min area")
max_area_1 = IntSlider(value=2000, min=50, max=30000, step=50, description="Max area")

# Buttons
save_button_1 = Button(description="💾 Save YOLO labels", button_style="success")


def run_part1(_=None):
    global current_stats, current_params, current_image_shape, current_image_name

    image_name = image_dropdown_1.value
    img_path = os.path.join(IMAGE_DIR, image_name)
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        with log_out:
            print(f"⚠️ Cannot read {img_path}")
        return

    H, W = gray.shape[:2]
    current_image_shape = (H, W)
    current_image_name = image_name

    mask_clean, stats, centroids, kept_count = segment_single_threshold(
        gray,
        thresh=thresh_1.value,
        bg_ksize=bg_ksize_1.value,
        blur_k=blur_k_1.value,
        invert=invert_1.value,
        min_area=min_area_1.value,
        max_area=max_area_1.value,
    )

    current_stats = stats
    current_params = dict(min_area=min_area_1.value, max_area=max_area_1.value)

    vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    num_labels = stats.shape[0]
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area_1.value or area > max_area_1.value:
            continue
        cx, cy = centroids[i]
        cx, cy = int(cx), int(cy)
        cv2.rectangle(vis, (x, y), (x + w, y + h), (255, 0, 0), 1)
        cv2.circle(vis, (cx, cy), 3, (0, 255, 0), -1)

    with out_part1:
        out_part1.clear_output(wait=True)
        plt.figure(figsize=(15, 5))
        plt.suptitle(
            f"[Single threshold] {image_name} | thresh={thresh_1.value} | kept objs: {kept_count}"
        )

        plt.subplot(1, 3, 1)
        plt.title("Original")
        plt.imshow(gray, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Mask")
        plt.imshow(mask_clean, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Result (bboxes)")
        plt.imshow(vis)
        plt.axis("off")

        plt.show()


def on_save_part1(_):
    save_bboxes_callback(_)


# Auto-run when parameters change
def _on_change_part1(change):
    if change["name"] == "value":
        run_part1()


for w in [image_dropdown_1, bg_ksize_1, blur_k_1, invert_1, thresh_1, min_area_1, max_area_1]:
    w.observe(_on_change_part1, names="value")


save_button_1.on_click(on_save_part1)

# Layout: group parameters into categories
preproc_box_1 = VBox(
    [HTML("<b>A. Preprocessing</b>"),
     bg_ksize_1, blur_k_1, invert_1]
)

thresh_box_1 = VBox(
    [HTML("<b>B. Thresholding</b>"),
     thresh_1]
)

filter_box_1 = VBox(
    [HTML("<b>C. Object filtering</b>"),
     min_area_1, max_area_1]
)

control_box_1 = VBox(
    [
        HTML("<b>Controls</b>"),
        image_dropdown_1,
        HBox([save_button_1]),
    ]
)

ui_part1 = VBox(
    [
        HTML("<h3>1️⃣ Manual single-threshold segmentation</h3>"),
        control_box_1,
        HBox([preproc_box_1, thresh_box_1, filter_box_1]),
        out_part1,
    ]
)

display(ui_part1)

# Initial run
run_part1()


In [ ]:
# 2️⃣ Manual multi-threshold stability map (no optimization)

# ---------- Core stability segmentation ----------

def segment_multi_threshold(gray,
                            t_min: int,
                            t_max: int,
                            n_steps: int,
                            min_hits: int,
                            bg_ksize: int,
                            blur_k: int,
                            invert: bool,
                            min_area: int,
                            max_area: int):
    """Multi-threshold stability segmentation.
    Returns: mask_clean, stats, centroids, kept_count, stability
    """
    H, W = gray.shape[:2]

    bg_ksize, blur_k, t_min, t_max, n_steps, min_hits = _sanitize_params(
        bg_ksize, blur_k, t_min, t_max, n_steps, min_hits
    )

    gray_blur = _preprocess_gray(gray, bg_ksize, blur_k)
    val = 255 - gray_blur if invert else gray_blur

    thresholds = np.linspace(t_min, t_max, n_steps).astype(np.uint8)
    stability = np.zeros_like(val, dtype=np.uint16)

    for t in thresholds:
        _, mask_t = cv2.threshold(val, int(t), 1, cv2.THRESH_BINARY)
        stability += mask_t.astype(np.uint16)

    mask_multi = np.zeros_like(val, dtype=np.uint8)
    mask_multi[stability >= min_hits] = 255

    kernel = np.ones((3, 3), np.uint8)
    mask_clean = cv2.morphologyEx(mask_multi, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel, iterations=1)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        mask_clean, connectivity=8
    )

    kept_count = 0
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area or area > max_area:
            continue
        kept_count += 1

    return mask_clean, stats, centroids, kept_count, stability


# ---------- UI (Part 2) ----------

out_part2 = Output()

image_dropdown_2 = Dropdown(options=image_names, description="Image")

# A. Preprocessing
bg_ksize_2 = IntSlider(value=31, min=3, max=101, step=2, description="BG ksize")
blur_k_2   = IntSlider(value=3,  min=1, max=31,  step=2, description="Blur k")
invert_2   = Checkbox(value=False, description="Invert")

# B. Stability (multi-threshold)
t_min_2    = IntSlider(value=60,  min=0,   max=254, step=1, description="t_min")
t_max_2    = IntSlider(value=160, min=1,   max=255, step=1, description="t_max")
n_steps_2  = IntSlider(value=8,   min=1,   max=30,  step=1, description="n_steps")
min_hits_2 = IntSlider(value=4,   min=1,   max=30,  step=1, description="min_hits")

# C. Object filtering
min_area_2 = IntSlider(value=50,   min=1,  max=5000,  step=10, description="Min area")
max_area_2 = IntSlider(value=2000, min=50, max=30000, step=50, description="Max area")

save_button_2 = Button(description="💾 Save YOLO labels", button_style="success")


def run_part2(_=None):
    global current_stats, current_params, current_image_shape, current_image_name

    image_name = image_dropdown_2.value
    img_path = os.path.join(IMAGE_DIR, image_name)
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        with log_out:
            print(f"⚠️ Cannot read {img_path}")
        return

    H, W = gray.shape[:2]
    current_image_shape = (H, W)
    current_image_name = image_name

    mask_clean, stats, centroids, kept_count, stability = segment_multi_threshold(
        gray,
        t_min=t_min_2.value, t_max=t_max_2.value,
        n_steps=n_steps_2.value, min_hits=min_hits_2.value,
        bg_ksize=bg_ksize_2.value, blur_k=blur_k_2.value, invert=invert_2.value,
        min_area=min_area_2.value, max_area=max_area_2.value,
    )

    current_stats = stats
    current_params = dict(min_area=min_area_2.value, max_area=max_area_2.value)

    vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    num_labels = stats.shape[0]
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area_2.value or area > max_area_2.value:
            continue
        cx, cy = centroids[i]
        cx, cy = int(cx), int(cy)
        cv2.rectangle(vis, (x, y), (x + w, y + h), (255, 0, 0), 1)
        cv2.circle(vis, (cx, cy), 3, (0, 255, 0), -1)

    if stability.max() > 0:
        stab_show = (stability.astype(np.float32) / stability.max()) * 255
        stab_show = stab_show.astype(np.uint8)
    else:
        stab_show = stability.astype(np.uint8)

    with out_part2:
        out_part2.clear_output(wait=True)
        plt.figure(figsize=(18, 5))
        plt.suptitle(
            f"[Stability] {image_name} | multi-th [{t_min_2.value}, {t_max_2.value}] "
            f"steps={n_steps_2.value}, min_hits={min_hits_2.value} | kept objs: {kept_count}"
        )

        plt.subplot(1, 3, 1)
        plt.title("Stability map")
        plt.imshow(stab_show, cmap="hot")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Final mask")
        plt.imshow(mask_clean, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Result (bboxes)")
        plt.imshow(vis)
        plt.axis("off")

        plt.show()


def on_save_part2(_):
    save_bboxes_callback(_)


def _on_change_part2(change):
    if change["name"] == "value":
        run_part2()


for w in [
    image_dropdown_2,
    bg_ksize_2, blur_k_2, invert_2,
    t_min_2, t_max_2, n_steps_2, min_hits_2,
    min_area_2, max_area_2,
]:
    w.observe(_on_change_part2, names="value")


save_button_2.on_click(on_save_part2)

preproc_box_2 = VBox(
    [HTML("<b>A. Preprocessing</b>"),
     bg_ksize_2, blur_k_2, invert_2]
)

stability_box_2 = VBox(
    [HTML("<b>B. Stability (multi-threshold)</b>"),
     t_min_2, t_max_2, n_steps_2, min_hits_2]
)

filter_box_2 = VBox(
    [HTML("<b>C. Object filtering</b>"),
     min_area_2, max_area_2]
)

control_box_2 = VBox(
    [
        HTML("<b>Controls</b>"),
        image_dropdown_2,
        HBox([save_button_2]),
    ]
)

ui_part2 = VBox(
    [
        HTML("<h3>2️⃣ Manual multi-threshold stability map</h3>"),
        control_box_2,
        HBox([preproc_box_2, stability_box_2, filter_box_2]),
        out_part2,
    ]
)

display(ui_part2)

# Initial run
run_part2()


In [ ]:
# 3️⃣ Stability map + local optimisation (interactive)

# ---------- Optimization over (t_min, t_max) ----------

def optimize_tmin_tmax(image_name,
                       tmin_range=(20, 80),
                       tmax_range=(60, 180),
                       step=10,
                       n_steps=8,
                       min_hits=4,
                       bg_ksize=31,
                       blur_k=3,
                       invert=False,
                       min_area=50,
                       max_area=2000):
    """Brute-force search for (t_min, t_max) that maximizes the number of kept boxes.
    Returns: best_tmin, best_tmax
    """
    img_path = os.path.join(IMAGE_DIR, image_name)
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        with log_out:
            print(f"⚠️ Cannot read {img_path} in optimization.")
        return None, None

    best_count = -1
    best_tmin = None
    best_tmax = None

    for t_min in range(tmin_range[0], tmin_range[1] + 1, step):
        for t_max in range(tmax_range[0], tmax_range[1] + 1, step):
            # enforce a minimal gap
            if t_max <= t_min + 5:
                continue

            _, _, _, kept_count, _ = segment_multi_threshold(
                gray,
                t_min=t_min, t_max=t_max,
                n_steps=n_steps, min_hits=min_hits,
                bg_ksize=bg_ksize, blur_k=blur_k, invert=invert,
                min_area=min_area, max_area=max_area,
            )

            if kept_count > best_count:
                best_count = kept_count
                best_tmin = t_min
                best_tmax = t_max

    with log_out:
        print(f"🔎 Optimization for {image_name}")
        print(f"  Best t_min = {best_tmin}, t_max = {best_tmax}, kept_count = {best_count}")

    return best_tmin, best_tmax


# ---------- UI (Part 3) ----------

out_part3 = Output()

image_dropdown_3 = Dropdown(options=image_names, description="Image")

# A. Preprocessing
bg_ksize_3 = IntSlider(value=31, min=3, max=101, step=2, description="BG ksize")
blur_k_3   = IntSlider(value=3,  min=1, max=31,  step=2, description="Blur k")
invert_3   = Checkbox(value=False, description="Invert")

# B. Stability (multi-threshold)
t_min_3    = IntSlider(value=60,  min=0,   max=254, step=1, description="t_min")
t_max_3    = IntSlider(value=160, min=1,   max=255, step=1, description="t_max")
n_steps_3  = IntSlider(value=8,   min=1,   max=30,  step=1, description="n_steps")
min_hits_3 = IntSlider(value=4,   min=1,   max=30,  step=1, description="min_hits")

# C. Object filtering
min_area_3 = IntSlider(value=50,   min=1,  max=5000,  step=10, description="Min area")
max_area_3 = IntSlider(value=2000, min=50, max=30000, step=50, description="Max area")

opt_button_3  = Button(description="🔍 Optimize t_min/t_max", button_style="info")
save_button_3 = Button(description="💾 Save YOLO labels", button_style="success")


def run_part3(_=None):
    """Run stability segmentation using current Part 3 widget values."""
    global current_stats, current_params, current_image_shape, current_image_name

    image_name = image_dropdown_3.value
    img_path = os.path.join(IMAGE_DIR, image_name)
    gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if gray is None:
        with log_out:
            print(f"⚠️ Cannot read {img_path}")
        return

    H, W = gray.shape[:2]
    current_image_shape = (H, W)
    current_image_name = image_name

    mask_clean, stats, centroids, kept_count, stability = segment_multi_threshold(
        gray,
        t_min=t_min_3.value, t_max=t_max_3.value,
        n_steps=n_steps_3.value, min_hits=min_hits_3.value,
        bg_ksize=bg_ksize_3.value, blur_k=blur_k_3.value, invert=invert_3.value,
        min_area=min_area_3.value, max_area=max_area_3.value,
    )

    current_stats = stats
    current_params = dict(min_area=min_area_3.value, max_area=max_area_3.value)

    vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    num_labels = stats.shape[0]
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area < min_area_3.value or area > max_area_3.value:
            continue
        cx, cy = centroids[i]
        cx, cy = int(cx), int(cy)
        cv2.rectangle(vis, (x, y), (x + w, y + h), (255, 0, 0), 1)
        cv2.circle(vis, (cx, cy), 3, (0, 255, 0), -1)

    if stability.max() > 0:
        stab_show = (stability.astype(np.float32) / stability.max()) * 255
        stab_show = stab_show.astype(np.uint8)
    else:
        stab_show = stability.astype(np.uint8)

    with out_part3:
        out_part3.clear_output(wait=True)
        plt.figure(figsize=(18, 5))
        plt.suptitle(
            f"[Stability + Opt] {image_name} | multi-th [{t_min_3.value}, {t_max_3.value}] "
            f"steps={n_steps_3.value}, min_hits={min_hits_3.value} | kept objs: {kept_count}"
        )

        plt.subplot(1, 3, 1)
        plt.title("Stability map")
        plt.imshow(stab_show, cmap="hot")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Final mask")
        plt.imshow(mask_clean, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Result (bboxes)")
        plt.imshow(vis)
        plt.axis("off")

        plt.show()


def on_optimize_part3(_):
    """Run optimization for current image and update t_min/t_max sliders, then rerun."""
    image_name = image_dropdown_3.value

    best_tmin, best_tmax = optimize_tmin_tmax(
        image_name,
        # Use the slider ranges as search windows:
        tmin_range=(t_min_3.min, t_min_3.max),
        tmax_range=(t_max_3.min, t_max_3.max),
        step=10,
        n_steps=n_steps_3.value,
        min_hits=min_hits_3.value,
        bg_ksize=bg_ksize_3.value,
        blur_k=blur_k_3.value,
        invert=invert_3.value,
        min_area=min_area_3.value,
        max_area=max_area_3.value,
    )

    if best_tmin is None or best_tmax is None:
        return

    t_min_3.value = best_tmin
    t_max_3.value = best_tmax

    # Rerun segmentation with optimized values
    run_part3()


def on_save_part3(_):
    save_bboxes_callback(_)


def _on_change_part3(change):
    if change["name"] == "value":
        run_part3()


for w in [
    image_dropdown_3,
    bg_ksize_3, blur_k_3, invert_3,
    t_min_3, t_max_3, n_steps_3, min_hits_3,
    min_area_3, max_area_3,
]:
    w.observe(_on_change_part3, names="value")


opt_button_3.on_click(on_optimize_part3)
save_button_3.on_click(on_save_part3)

preproc_box_3 = VBox(
    [HTML("<b>A. Preprocessing</b>"),
     bg_ksize_3, blur_k_3, invert_3]
)

stability_box_3 = VBox(
    [HTML("<b>B. Stability (multi-threshold)</b>"),
     t_min_3, t_max_3, n_steps_3, min_hits_3]
)

filter_box_3 = VBox(
    [HTML("<b>C. Object filtering</b>"),
     min_area_3, max_area_3]
)

control_box_3 = VBox(
    [
        HTML("<b>Controls</b>"),
        image_dropdown_3,
        HBox([opt_button_3, save_button_3]),
    ]
)

ui_part3 = VBox(
    [
        HTML("<h3>3️⃣ Stability map + (t_min, t_max) optimization</h3>"),
        control_box_3,
        HBox([preproc_box_3, stability_box_3, filter_box_3]),
        out_part3,
    ]
)

display(ui_part3)

# Initial run
run_part3()


In [5]:
# 4️⃣ Full automatic optimization & labeling pipeline (no UI)

def optimize_and_label_all_images(
    tmin_range=(20, 80),
    tmax_range=(60, 180),
    step=10,
    n_steps=8,
    min_hits=4,
    bg_ksize=31,
    blur_k=3,
    invert=True,
    min_area=50,
    max_area=2000,
):
    """Full pipeline: per-image optimization + YOLO label export.
    All `.txt` labels are written into LABEL_DIR (one per image).
    """
    os.makedirs(LABEL_DIR, exist_ok=True)

    total_images = len(image_paths)
    with log_out:
        print(f"🚀 Batch optimization & labeling on {total_images} images")

    for idx, img_path in enumerate(image_paths):
        image_name = os.path.basename(img_path)
        with log_out:
            print(f"[{idx+1}/{total_images}] Processing {image_name}")

        best_tmin, best_tmax = optimize_tmin_tmax(
            image_name,
            tmin_range=tmin_range,
            tmax_range=tmax_range,
            step=step,
            n_steps=n_steps,
            min_hits=min_hits,
            bg_ksize=bg_ksize,
            blur_k=blur_k,
            invert=invert,
            min_area=min_area,
            max_area=max_area,
        )

        if best_tmin is None or best_tmax is None:
            with log_out:
                print("  ⚠️ Skipping (no valid parameters found)")
            continue

        gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if gray is None:
            with log_out:
                print(f"  ⚠️ Cannot read {img_path}, skipping")
            continue

        mask_clean, stats, centroids, kept_count, stability = segment_multi_threshold(
            gray,
            t_min=best_tmin,
            t_max=best_tmax,
            n_steps=n_steps,
            min_hits=min_hits,
            bg_ksize=bg_ksize,
            blur_k=blur_k,
            invert=invert,
            min_area=min_area,
            max_area=max_area,
        )

        H, W = gray.shape[:2]
        stem, _ = os.path.splitext(image_name)
        label_path = os.path.join(LABEL_DIR, stem + ".txt")

        kept = 0
        with open(label_path, "w") as f:
            for i in range(1, stats.shape[0]):
                x, y, w, h, area = stats[i]
                if area < min_area or area > max_area:
                    continue
                kept += 1

                cx = x + w / 2.0
                cy = y + h / 2.0

                x_norm = cx / W
                y_norm = cy / H
                w_norm = w / W
                h_norm = h / H

                f.write(f"0 {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        with log_out:
            print(f"  ✅ Saved {kept} boxes to {label_path}")

    with log_out:
        print("✨ Done: all images processed.")


display(HTML("<h3>4️⃣ Full automatic optimization & labeling (no UI)</h3>"))



optimize_and_label_all_images(
    tmin_range=(20, 80),
    tmax_range=(60, 180),
    step=30,
    n_steps=30,
    min_hits=4,
    bg_ksize=31,
    blur_k=3,
    invert=False,
    min_area=50,
    max_area=500,
)


HTML(value='<h3>4️⃣ Full automatic optimization & labeling (no UI)</h3>')